Importing Dependencies

In [ ]:
import cv2
import time
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import os

import pandas as pd
import torch
import torch.nn as nn
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.utils import to_categorical

from tqdm import tqdm
from keras import regularizers

Defining necessary functions

In [ ]:
#Handland takes the mediapipe output landmarks, collates all the coordinates and flattens it to create an array of size 63

def handland(results):
    hl = np.array([[landmark.x,landmark.y,landmark.z] for landmark in results.multi_hand_landmarks[0].landmark]).flatten() if results.multi_hand_landmarks else np.zeros(63)
    return hl

def handscore(results):
    sc = results.multi_handedness[0].classification[0].score
    return sc

#Choose is basically the combined model. We have three different custom models and the combination of the three seems to be working better.

def choose(a,b,c):
    if a != '':
        a = ord(a)
        b = ord(b)
        c = ord(c)
        if (b-c) == 0:
            return chr(b)
        else:
            return chr(a)
    else:
        return ''


#Augmentation Function

from PIL import Image as PILImage
def augment_image(image):
    # Read the original image
    original_image = image

    # Convert image from BGR to RGB
    original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_RGB2BGR)

    # Initialize the list to store augmented images
    augmented_images = [original_image_rgb]

    # Flip the image horizontally
    # flipped_image = cv2.flip(original_image_rgb, 1)
    # augmented_images.append(flipped_image)

    # Rotate the image by custom angles
    rotation_angles = np.arange(-15,16,10)
    for angle in rotation_angles:
        rotated_image = PILImage.fromarray(original_image_rgb)
        rotated_image = rotated_image.rotate(angle)
        rotated_image = np.array(rotated_image)
        augmented_images.append(rotated_image)

    # GaussianBlur the image

    blurred_image = cv2.GaussianBlur(original_image_rgb, (5 , 5), 0)
    augmented_images.append(blurred_image)

    return augmented_images

Mediapipe Functions

In [ ]:
# Grabbing the Holistic Model from Mediapipe and
# Initializing the Model
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
	min_detection_confidence=0.1,
	min_tracking_confidence=0.1
)

# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles

Loading the model weights

In [ ]:
#We load the weights for all the 3 custom models here

#The efficiency of the model is decreasing with increase in model number i.e. Model 1 is suppposed to be the best and Model 3 is worst
#Model 1 is trained on the custom train dataset we created
#Model 2 is trained on the Augmented Dataset we obtained from the Yolo  Train Data
#Model 3 is trained on a much bigger dataset + augmentation, but the quality of the dataset was not up to the mark and it often used wrong signs

def baseline_model():
  model = Sequential()
  model.add(Dense(46, input_dim=63, activation='relu', kernel_regularizer=regularizers.l1(0.01)))
  model.add(Dense(26, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model1 = baseline_model()
model1.load_weights("wcdata_2layers.h5")
model2 = baseline_model()
model2.load_weights("augdata_2layer.h5")
model3 = baseline_model()
model3.load_weights("bigdataASLNN26.h5")

Loading custom test dataset

In [ ]:
Let = np.loadtxt('custom_test_data_mp.txt')
Testlabel = np.loadtxt('custom_test_labels.txt',dtype = str)

Evaluating the models on the test dataset

In [ ]:
pred1 = [chr(np.argmax(i)+65) for i in model1(Let)]
pred2 = [chr(np.argmax(i)+65) for i in model2(Let)]
pred3 = [chr(np.argmax(i)+65) for i in model3(Let)]
pred4 = [choose(pred1[i],pred2[i],pred3[i]) for i in range(len(Let))]

Confusion Matrix Calculations

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm1 = confusion_matrix(Testlabel,pred1)
cm2 = confusion_matrix(Testlabel,pred2)
cm3 = confusion_matrix(Testlabel,pred3)
cm4 = confusion_matrix(Testlabel,pred4)

plt.title('Model 3')
sns.heatmap(cm3)
plt.show()
plt.title('Model 2')
sns.heatmap(cm2)
plt.show()
plt.title('Model 1')
sns.heatmap(cm1)
plt.show()
plt.title('Combined Model')
sns.heatmap(cm4)

Accuracy Calculation

In [ ]:
acc1,acc2,acc3,acc4 = 0,0,0,0

for i in range(len(Testlabel)):
    if pred1[i] == Testlabel[i]:
        acc1+=1
    if pred2[i] == Testlabel[i]:
        acc2+=1
    if pred3[i] == Testlabel[i]:
        acc3+=1
    if pred4[i] == Testlabel[i]:
        acc4+=1
acc1/= len(Testlabel)
acc2/= len(Testlabel)
acc3/= len(Testlabel)
acc4/= len(Testlabel)

acc1,acc2,acc3,acc4

Recall Calculation

In [ ]:
r1,r2,r3,r4 = 0,0,0,0
for i in range(len(cm1)):
    r1 += (cm1[i,i]/np.sum(cm1[i]))
    r2 += (cm2[i,i]/np.sum(cm2[i]))
    r3 += (cm3[i,i]/np.sum(cm3[i]))
    r4 += (cm4[i,i]/np.sum(cm4[i]))
r3,r2,r1,r4 = r3/26,r2/26,r1/26,r4/26

Precision Calculation

In [ ]:
p1,p2,p3,p4 = 0,0,0,0
cm1 = cm1.T
cm2 = cm2.T
cm3 = cm3.T
cm4 = cm4.T
for i in range(len(cm1)):
    p1 += (cm1[i,i]+0.001)/(np.sum(cm1[i])+0.001*len(cm1[i]))
    p2 += (cm2[i,i]+0.001)/(np.sum(cm2[i])+0.001*len(cm2[i]))
    p3 += (cm3[i,i]+0.001)/(np.sum(cm3[i])+0.001*len(cm3[i]))
    p4 += (cm4[i,i]/np.sum(cm4[i]))
p3,p2,p1,p4 = p3/26,p2/26,p1/26,p4/26

F-1 score calculation

In [ ]:
f1,f2,f3,f4 = (2*p1*r1)/(p1+r1), (2*p2*r2)/(p2+r2), (2*p3*r3)/(p3+r3), (2*p4*r4)/(p4+r4)
f3,f2,f1,f4

Printing  Scores like a table

In [ ]:
print("\t \t Accuracies: \t\t Average Precision: \t\t Average Recall \t\t Averafe F-1 score")
print( " Model 3: \t",round(acc3,3),"\t\t\t",round(p3,3),"\t\t\t\t",round(r3,3),"\t\t\t\t",round(f3,3),
      "\n Model 2: \t",round(acc2,3),"\t\t\t",round(p2,3),"\t\t\t\t",round(r2,3),"\t\t\t\t",round(f2,3),
      "\n Model 1: \t",round(acc1,3),"\t\t\t",round(p1,3),"\t\t\t\t",round(r1,3),"\t\t\t\t",round(f1,3),
      "\n Combined: \t", round(acc4,3),"\t\t\t",round(p4,3),"\t\t\t\t",round(r4,3),"\t\t\t\t",round(f4,3))